**1- Install Modules:**

install ultralytics for YOLOv8

In [ ]:
#-- Install-----------------------------------------------------------------------------------------------------
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
#---------------------------------------------------------------------------------------------------------------

**2- Import Modules:**

In [ ]:
#-- Import -----------------------------------------------------------------------------------------------------
from ultralytics import YOLO
import yaml

import torch

import numpy as np
import pandas as pd

import cv2
from IPython import display
import matplotlib.pyplot as plt

import shutil
import os

import random
#---------------------------------------------------------------------------------------------------------------

**3- Download Pothole Dataset:**

ds linke : https://public.roboflow.com/object-detection/pothole

Note: sign up on roboflow.com and obtain your API_KEY for usage.

In [ ]:
#-- Download Pothole Dataset From Roboflow  ----------------------------------------------------------------------
!curl -L 'https://public.roboflow.com/ds/YOUR_API_KEY' > roboflow.zip
!unzip -o roboflow.zip
!rm roboflow.zip

display.clear_output()
print('DS successfully downloaded :)')
#---------------------------------------------------------------------------------------------------------------

**4- Initialize Params:**

This project was written and executed on kaggle.com. If you choose to run it in a different environment, please ensure to configure the output_path variable according to your specific path.

In [ ]:
#-- Initialize -------------------------------------------------------------------------------------------------
output_path = '/kaggle/working/'

train_img_dir = output_path + 'train/images/'
val_img_dir = output_path + 'valid/images/'
test_img_dir = output_path + 'test/images/'

train_lbl_dir = output_path + 'train/labels/'
val_lbl_dir = output_path + 'valid/labels/'
test_lbl_dir = output_path + 'test/labels/'

NUM_EPOCHS = 50

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:' , DEVICE)

data_config_file = 'data.yaml'

CONF_THRESHOLDS = [0.25,0.3,0.4,0.5,0.6,0.75,0.9]
IOU_THRESHOLDS = [0.5,0.6,0.7,0.8] 
NUMBER_OF_FROZEN_LAYERS = [0,2,4,5,7,10,12,15,17,20,22]

plots_dir = output_path + 'plots/' 
if shutil.os.path.exists(plots_dir):
    shutil.rmtree(plots_dir)
shutil.os.makedirs(plots_dir)
#---------------------------------------------------------------------------------------------------------------

**5- Retrieve the sizes of the training, validation, and test datasets.**

In [ ]:
#-- DS Size ----------------------------------------------------------------------------------------------------
train_size = len([name for name in os.listdir(train_img_dir) if os.path.isfile(os.path.join(train_img_dir, name))])
val_size = len([name for name in os.listdir(val_img_dir) if os.path.isfile(os.path.join(val_img_dir, name))])
test_size = len([name for name in os.listdir(test_img_dir) if os.path.isfile(os.path.join(test_img_dir, name))])

print(f'train size:{train_size}\nvalidation size:{val_size}\ntest size:{test_size}')
#---------------------------------------------------------------------------------------------------------------

**6- Present 5 images from the training dataset next to images with bounding boxes:**

We have defined two functions:
* draw_bboxes: A function to draw bounding boxes on an image.
* read_bboxes: A function to read and convert bounding box coordinates.

In [ ]:
# Function to draw bounding boxes on an image ------------------------------------------------------------------
def draw_bboxes(image, bboxes):
    for bbox in bboxes:
        x, y, w, h = bbox
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return image
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to read and convert bounding box coordinates ------------------------------------------------------
def read_bboxes(file_path, img_shape):
    with open(file_path, 'r') as file:
        bboxes = []
        for line in file:
            # YOLO format: class, x_center, y_center, width, height (normalized)
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x_center, y_center = x_center * img_shape[1], y_center * img_shape[0]
            width, height = width * img_shape[1], height * img_shape[0]
            x, y = int(x_center - width / 2), int(y_center - height / 2)
            bboxes.append([int(x), int(y), int(width), int(height)])
    return bboxes
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- show 5 random images from our training set next to labeled images with bounding boxes ----------------------
#-- Get a list of image file names --
image_files = [f for f in os.listdir(train_img_dir) if f.endswith('.jpg')]
random.shuffle(image_files)

#-- Select 5 random images --
selected_images = image_files[:5]

#-- Create subplots --
fig, axes = plt.subplots(5, 2, figsize=(10, 20))
fig.suptitle('Original and Labeled Images\n')

for i, img_file in enumerate(selected_images):
    img_path = os.path.join(train_img_dir, img_file)
    bbox_path = os.path.join(train_lbl_dir, os.path.splitext(img_file)[0] + '.txt')

    #-- Read image --
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #-- Convert from BGR to RGB
    img_shape = image.shape

    #-- Read and draw bounding boxes --
    bboxes = read_bboxes(bbox_path, img_shape)
    image_with_bboxes = draw_bboxes(image.copy(), bboxes)

    #-- Show original and labeled images --
    axes[i, 0].imshow(image)
    axes[i, 0].axis('off')
    axes[i, 0].set_title('Original')

    axes[i, 1].imshow(image_with_bboxes)
    axes[i, 1].axis('off')
    axes[i, 1].set_title('Image with bounding box')

plt.tight_layout()
plt.savefig( plots_dir + 'Original_and_Labeled_Images.png')
plt.show()
#---------------------------------------------------------------------------------------------------------------

**7- Modify the configuration file if it is necessary:**

We haven't altered the paths for the train, test, and validation datasets, so there's no need to modify the configuration file (data.yaml).

We intend to execute the validation function on the test data. Therefore, we have created a new version of the configuration file (test_data.yaml).



In [ ]:
#-- Configuring the .yaml file for evaluating model on test data -----------------------------------------------
test_data_config_file = output_path + 'test_data.yaml'

config = {'train': '../train/images',
         'val': '../test/images',
         'nc': 1,
         'names': ['drone']}

with open(test_data_config_file, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)
#---------------------------------------------------------------------------------------------------------------

**8- Load YOLO Model and Train**

In [ ]:
#-- Train Model and Save Results --------------------------------------------------------------------------
def train_model(model, number_of_frozen_layers, num_of_epochs):   
    
    #--  set project and name to create new folder for each run --
    project_value = 'model_' + str(number_of_frozen_layers) + '_frozen'   
    name_value = 'train'
    
    #-- Train --
    print(f'\n\nTraining Model - {number_of_frozen_layers} frozen layers  ********************************\n\n')
    model.train(data = data_config_file,
                epochs = num_of_epochs,
                freeze = number_of_frozen_layers,
                device = DEVICE,
                val = True,
                save = True,
                exist_ok = True,
                plots=True,
                project = project_value,
                name = name_value)
#---------------------------------------------------------------------------------------------------------------    

In [ ]:
#-- Train Model With Different frozen Layers and save Results -------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS:
    #-- load YOLO model --
    model = YOLO("yolov8m.pt") 
    
    train_model(model = model,
                number_of_frozen_layers = n_frozen,
                num_of_epochs = NUM_EPOCHS)
    

#-- clear output
display.clear_output()
print('Finished training :)')
#---------------------------------------------------------------------------------------------------------------

**9- Plot different Training Curves:**

In [ ]:
#-- Plot Results Curves -----------------------------------------------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS:    
    curves_path = output_path + 'model_' + str(n_frozen) + '_frozen/train/'   
    
    curve_file = curves_path + 'results.png'    
    curve = cv2.cvtColor(cv2.imread(curve_file),cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(15,10))
    plt.title(f'Results - {n_frozen} frozen Layers')
    plt.axis('off')
    plt.savefig( plots_dir + 'results_' + str(n_frozen) + '.png')
    plt.imshow(curve)
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Plot Precision_Recall Curves --------------------------------------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS:    
    curves_path = output_path + 'model_' + str(n_frozen) + '_frozen/train/'   
    
    curve_file = curves_path + 'PR_curve.png'    
    curve = cv2.cvtColor(cv2.imread(curve_file),cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8,5))
    plt.title(f'PR - {n_frozen} frozen Layers')
    plt.axis('off')
    plt.savefig( plots_dir + 'PR_' + str(n_frozen) + '.png')
    plt.imshow(curve)   
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Plot Precision and Recall Curves ----------------------------------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS:    
    curves_path = output_path + 'model_' + str(n_frozen) + '_frozen/train/'   
    
    p_curve_file = curves_path + 'P_curve.png'    
    r_curve_file = curves_path + 'R_curve.png'   
    
    p_curve = cv2.cvtColor(cv2.imread(p_curve_file),cv2.COLOR_BGR2RGB)
    r_curve = cv2.cvtColor(cv2.imread(r_curve_file),cv2.COLOR_BGR2RGB) 
    
    fig, (axs1, axs2) = plt.subplots(1, 2, figsize = (10, 8))    
    axs1.title.set_text(f'Precison - {n_frozen} Layers frozen')
    axs1.axis('off')
    axs1.imshow(p_curve)   
    
    axs2.title.set_text(f'Recall - {n_frozen} frozen Layers')
    axs2.axis('off')
    plt.savefig( plots_dir + 'P_and_R_' + str(n_frozen) + '.png')
    axs2.imshow(p_curve)  
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Plot Confusion Matrix --------------------------------------------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS:    
    curves_path = output_path + 'model_' + str(n_frozen) + '_frozen/train/'   
    
    cm_file = curves_path + 'confusion_matrix.png'
    cm = cv2.cvtColor(cv2.imread(cm_file),cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(25,20))
    plt.title(f'Confusion Matrix - {n_frozen} frozen Layers')
    plt.axis('off')
    plt.savefig( plots_dir + 'train_cm_' + str(n_frozen) + '.png')
    plt.imshow(cm)   
#---------------------------------------------------------------------------------------------------------------

**10- Evaluate Model on Validation DS:**

In [ ]:
#-- Evaluate Model on Val Data ---------------------------------------------------------------------------------
def val_model(model, NUMBER_OF_FROZEN_LAYERS):
    
    project_value = 'model_' + str(NUMBER_OF_FROZEN_LAYERS) + '_frozen'   
    name_valu = 'validation'
    
    #-- Val --
    print(f'\n\nEvaluating Model on Val Data - {NUMBER_OF_FROZEN_LAYERS} frozen layers *******************\n\n')
    metrics = model.val(data = data_config_file,
                        device = DEVICE,
                        project = project_value,
                        name = name_valu)

    map_50_95 = metrics.box.map
    map_50 = metrics.box.map50
    map_75 = metrics.box.map75
    maps = metrics.box.maps #--a list contains map50-95 of each category --
    
    msg = f'''\n\n-----------------------------------------------
                \nmap_50_95:{map_50_95}\nmap_50:{map_50}\nmap_75:{map_75}\nmaps:{maps}
                \n-----------------------------------------------\n\n
            '''
    print(msg)
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Val Models With Different frozen Layers and save Results -------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS:
    
    #-- load best model --
    best_model_file = output_path + 'model_' + str(n_frozen) + '_frozen' + '/train/weights/best.pt'
    best_model = YOLO(best_model_file)     
    
    val_model(model= best_model,
              NUMBER_OF_FROZEN_LAYERS = n_frozen)
#---------------------------------------------------------------------------------------------------------------

**11- Evaluate Model On Test DS:**

In [ ]:
#-- Evaluate Model on Test Data ---------------------------------------------------------------------------------
def test_model(model, number_of_frozen_layers, conf , iou):
    
    project_value = 'model_' + str(number_of_frozen_layers) + '_frozen'   
    name_valu = 'test/test_' + str(conf) + '_' + str(iou)
    
    #-- Val --
    log = f'''
            \n\nEvaluating Model on Test Data - {number_of_frozen_layers} frozen layers *******************
            \n--- conf = {conf} - iou = {iou} ---\n
            '''
    print(log)
    
    metrics = model.val(data = test_data_config_file,
                        conf = conf,
                        iou = iou,
                        device = DEVICE,
                        project = project_value,
                        name = name_valu)

    map_50_95 = metrics.box.map
    map_50 = metrics.box.map50
    map_75 = metrics.box.map75
    maps = metrics.box.maps #--a list contains map50-95 of each category --
    
    msg = f'''\n-----------------------------------------------
                \nmap_50_95:{map_50_95}\nmap_50:{map_50}\nmap_75:{map_75}\nmaps:{maps}
                \n-----------------------------------------------\n
            '''
    print(msg)
    
    #-- Save metrics --
    results_file = 'model_' + str(number_of_frozen_layers) + '_results.csv'
    df_results = pd.read_csv(results_file)    
            
    results = {'conf':conf,
               'iou':iou,
               'map_50_95':map_50_95,
               'map_50':map_50,
               'map_75':map_75,
               'maps':maps}

    temp_df = pd.DataFrame(results)
    df_results = pd.concat([df_results, temp_df], ignore_index=True)  
    df_results.to_csv(output_path + results_file, index=False)    
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Test Models With Different frozen Layers and save Results -------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS:
    
    #-- load best model --
    best_model_file = output_path + 'model_' + str(n_frozen) + '_frozen' + '/train/weights/best.pt'
    best_model = YOLO(best_model_file)     
    
    #-- Create Empty DF to save results --
    cols_names = ['conf', 'iou', 'map_50_95', 'map_50', 'map_75', 'maps']        
    df_results = pd.DataFrame(columns=cols_names)
    results_file = 'model_' + str(n_frozen) + '_results.csv'
    df_results.to_csv(output_path + results_file, index=False)    
    
    for conf in CONF_THRESHOLDS:
        for iou in IOU_THRESHOLDS:   
            test_model(model = best_model,
                       number_of_frozen_layers = n_frozen,
                       conf = conf,
                       iou = iou)            
#---------------------------------------------------------------------------------------------------------------

**12: Plot Confusion Matrix for Test DS:**

In [ ]:
#-- Plot Confusion Matrix for Test Data ---------------------------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS: 
    for conf in CONF_THRESHOLDS:
        for iou in IOU_THRESHOLDS:   
    
            cms_path = output_path + 'model_' + str(n_frozen) + '_frozen/test/test_' + str(conf) + '_' + str(iou) + '/'
            cm_file = cms_path + 'confusion_matrix.png'            
            cm = cv2.cvtColor(cv2.imread(cm_file),cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(25,20))
            plt.title(f'Confusion Matrix - {n_frozen} Layers frozen - conf={conf} - iou={iou}')
            plt.axis('off')
            plt.savefig( plots_dir + 'test_cm_' + str(n_frozen) + '_' + str(conf) + '_' + str(iou) + '.png')
            plt.imshow(cm)   
#------------------------------------------------------------------------------------------------------------

**13-1: Plot and Compare mAP on Test Data:**

In [ ]:
#-- Plot and Compare mAP on Test Data  --------------------------------------------------------------------------------
def plot_mAP_results(number_of_frozen_layers):
    
    #-- load results --
    df_file = 'model_' + str(number_of_frozen_layers) + '_results.csv'
    df = pd.read_csv(output_path + df_file)
    
    #-- list of metrics --
    metrics = ['map_50_95', 'map_50', 'map_75']
    
    #-- create figure --
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f'mAP - {number_of_frozen_layers} frozen Layers\n')
    
    #-- plot bars for all metrics --
    i = 0
    for mtr in metrics:    
        sub_df = df[['conf' , 'iou' , mtr]]   
        sub_df['x_ticks'] = 'conf:' + sub_df['conf'].astype(str) + '- IoU:' +  sub_df['iou'].astype(str)        
        
        bars = axes[i].bar(sub_df['x_ticks'], sub_df[mtr], color = 'blue')    
        
        max_index = sub_df[mtr].idxmax()
        max_value = sub_df[mtr].max() 
        
        #-- Set a different color for the bar with the maximum value --
        bars[max_index].set_color('green')
        axes[i].text(max_index, max_value, f'Max: {round(max_value,4)}', ha='center', va='bottom')
        
        axes[i].set_title(mtr)           
        axes[i].set_xticklabels(labels = sub_df['x_ticks'], rotation=90) 
        
        i = i+1

    plt.savefig(plots_dir + 'mAP_'+ str(number_of_frozen_layers) +  '.png', bbox_inches='tight')
    plt.show()    
#------------------------------------------------------------------------------------------------------------   

In [ ]:
#-- Plot and Compare mAP on Test Data for all Models ------------------------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS: 
    plot_mAP_results(n_frozen)
#----------------------------------------------------------------------------------------------------------------------

**13-2: Assess the impact of the number of frozen layers on the model's performance:**

In [ ]:
#-- Plot and Compare Best Results ------------------------------------------------------------------------------------
def plot_best_mAP_results():
    
    #-- list of metrics --
    metrics = ['map_50', 'map_75', 'map_50_95']
    
    #-- get best results for each model --
    reults = {}
    for m in metrics:
        reults[m] = []
        for n in NUMBER_OF_FROZEN_LAYERS:
            
            df_file = 'model_' + str(n) + '_results.csv'
            df = pd.read_csv(df_file)           

            max_index = df[m].idxmax()
            max_value = df[m].max()  
            reults[m].append(max_value)            

    #-- create figure --
    fig, axes = plt.subplots(3, 1, figsize=(8, 15))
    fig.suptitle('Comparing the effects of the number of frozen layers')
    
    #-- plot bars --
    i = 0
    for metric , values in reults.items():
        y = values
        x = [str(l) for l in NUMBER_OF_FROZEN_LAYERS]         
        
        bars = axes[i].bar(x, y)
        
        max_value = max(y)
        max_index = y.index(max_value) 
        bars[max_index].set_color('green')
        
        axes[i].text(max_index, max_value, f'Max: {round(max_value,4)}', ha='center', va='bottom')
        
        axes[i].set_title(metric)           
        axes[i].set_xticklabels(labels = x) 
        
        i += 1

    plt.savefig(plots_dir + 'compare_best_mAP_values.png', bbox_inches='tight')
    plt.show()   
#----------------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Plot and Compare Best Results for Models with different frozen Layers ------------------------------------------------
plot_best_mAP_results()
#----------------------------------------------------------------------------------------------------------------------

**14- Make predictions in test images.:**

In [ ]:
#-- Get best conf and iou values for map_50 for each model ----------------------------------------------------------------
metric = 'map_50'

#-- create dict: key = NUMBER_OF_FROZEN_LAYERS , value = (conf , iou) for best map_50 --
best_results = {}

for n_frozen in NUMBER_OF_FROZEN_LAYERS:
    
    df_file = 'model_' + str(n_frozen) + '_results.csv'
    df = pd.read_csv(df_file) 
    
    sub_df = df[['conf' , 'iou' , metric]]        
    max_index = sub_df[metric].idxmax()    
    max_conf = sub_df.loc[max_index, 'conf']
    max_iou = sub_df.loc[max_index, 'iou'] 
    
    best_results[n_frozen] = (max_conf,max_iou)
#----------------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Predict Drone Objects on Test Images ----------------------------------------------------------------------------------
for n_frozen in NUMBER_OF_FROZEN_LAYERS:   
    
    #-- load best model --
    best_model_file = output_path + 'model_' + str(n_frozen) + '_frozen' + '/train/weights/best.pt'
    best_model = YOLO(best_model_file)       
    
    #-- get conf and iou for best map_50 --
    max_conf = best_results[n_frozen][0]
    max_iou = best_results[n_frozen][1]
    
    #-- set project and name for creating result dir --
    project_value = 'model_' + str(n_frozen) + '_frozen/predictions'   
    name_value = 'predictions_' + str(max_conf) + '_' + str(max_iou)      
      
    best_model.predict(source = test_img_dir,
                       conf = max_conf,
                       iou = max_iou,
                       show = False,
                       save= True,
                       project= project_value,
                       name=name_value) 
display.clear_output()
print('Finished Predicting :)')
#---------------------------------------------------------------------------------------------------------------

**15- Visualize five images with ground truth labels alongside predicted objects using various trained models.**

In [ ]:
#-- Plot a Ground Truth Image next to All Predicted Images ------------------------------------------------------
def plot_images(images, file_name):
    
    #-- create figure --
    fig, axes = plt.subplots(6, 2, figsize=(12, 20))

    #-- Flatten the 6x4 array of axes into a 1D array --
    axes = axes.flatten()
    
    for i, (img, ax) in enumerate(zip(images, axes)):
        ax.imshow(img)         #ax.imshow(img, cmap='gray')  # Assuming grayscale images        
        ax.set_xticks([])
        ax.set_yticks([])
        
        title = ''
        if i==0:
            title = 'Ground Truth'
        else:
            title = f'Predicted - {NUMBER_OF_FROZEN_LAYERS[i-1]} frozen layers'

        ax.set_title(title)

    
    plt.tight_layout()
    
    plt.savefig(file_name, bbox_inches='tight')
    plt.show()
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Plot Ground Truth and Predictions for 5 random Images from Test Data ---------------------------------------
#-- Get a list of image file names in Test Dir --
image_files = [f for f in os.listdir(test_img_dir) if f.endswith('.jpg')]
random.shuffle(image_files)

#-- Select 5 random images --
selected_images = image_files[:5]

for img_file in selected_images:
    
    #-- set path --
    img_path = os.path.join(test_img_dir, img_file)
    gt_bbox_path = os.path.join(test_lbl_dir, os.path.splitext(img_file)[0] + '.txt')
    
    #-- load image --
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_shape = img.shape

    #-- Read and draw bounding boxes --
    gt_bboxes = read_bboxes(gt_bbox_path, img_shape)
    gt_image_with_bboxes = draw_bboxes(img.copy(), gt_bboxes)

    #-- create list of ground truth image and all predictions --
    images = []
    images.append(gt_image_with_bboxes)

    for n_frozen in NUMBER_OF_FROZEN_LAYERS:    

        #-- get conf and iou for best map_50 --
        max_conf = best_results[n_frozen][0]
        max_iou = best_results[n_frozen][1]

        #-- set path --
        project_value = 'model_' + str(n_frozen) + '_frozen/predictions/'   
        name_value = 'predictions_' + str(max_conf) + '_' + str(max_iou) 
        pred_img_path = project_value + name_value + '/'
        pred_img_file = os.path.join(pred_img_path, img_file)

        #-- load predicted images --
        pred_img = cv2.imread(pred_img_file)
        pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB) 

        images.append(pred_img)  
    
    file_name = plots_dir + 'predictions_' + str(img_file) + '.png'
    plot_images(images, file_name)
#---------------------------------------------------------------------------------------------------------------